In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install gradio soundfile numpy

import os
import gradio as gr
import soundfile as sf
import numpy as np
import tempfile

# Root folder of separated stems (MUSDB18HQ test set)
ROOT_DIR = "/content/drive/MyDrive/musdb18hq/test"


def list_songs():
    songs = []
    for name in os.listdir(ROOT_DIR):
        full = os.path.join(ROOT_DIR, name)
        if os.path.isdir(full):
            has_all = all(
                os.path.exists(os.path.join(full, stem + ".wav"))
                for stem in ["vocals", "drums", "bass", "other"]
            )
            if has_all:
                songs.append(name)
    songs.sort()
    return songs


SONG_LIST = list_songs()
if not SONG_LIST or len(SONG_LIST) < 2:
    raise RuntimeError(
        "Need at least 2 songs with 4 stems under ROOT_DIR. Check the ROOT_DIR path."
    )


def ensure_stereo(x):
    """Convert mono to stereo if needed."""
    if x.ndim == 1:
        x = np.stack([x, x], axis=1)
    return x


def load_stems(song_name):
    folder = os.path.join(ROOT_DIR, song_name)
    vocals, sr = sf.read(os.path.join(folder, "vocals.wav"))
    drums, _ = sf.read(os.path.join(folder, "drums.wav"))
    bass, _ = sf.read(os.path.join(folder, "bass.wav"))
    other, _ = sf.read(os.path.join(folder, "other.wav"))

    vocals = ensure_stereo(vocals)
    drums = ensure_stereo(drums)
    bass = ensure_stereo(bass)
    other = ensure_stereo(other)

    min_len = min(len(vocals), len(drums), len(bass), len(other))
    vocals = vocals[:min_len]
    drums = drums[:min_len]
    bass = bass[:min_len]
    other = other[:min_len]

    return vocals, drums, bass, other, sr


def remix_two_core(song_a, song_b, g_vocals, g_drums, g_bass, g_other):
    """Mix two songs: vocals+bass from A, drums+other from B."""
    if song_a == song_b:
        raise ValueError("Please choose two different songs.")

    vocals_a, drums_a, bass_a, other_a, sr_a = load_stems(song_a)
    vocals_b, drums_b, bass_b, other_b, sr_b = load_stems(song_b)

    # use overlapping length of the selected stems
    seg_len = min(
        len(vocals_a),
        len(drums_b),
        len(bass_a),
        len(other_b),
    )

    vocals = vocals_a[:seg_len]
    drums = drums_b[:seg_len]
    bass = bass_a[:seg_len]
    other = other_b[:seg_len]

    mix = (
        g_vocals * vocals
        + g_drums * drums
        + g_bass * bass
        + g_other * other
    )

    if mix.size == 0:
        raise RuntimeError("Empty mix generated. Check stems for these songs.")

    max_val = np.max(np.abs(mix))
    if max_val > 1e-8 and max_val > 1:
        mix = mix / max_val

    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp.name, mix, sr_a)
    return tmp.name


def manual_two_song_remix(song_a, song_b, g_vocals, g_drums, g_bass, g_other):
    if not song_a or not song_b:
        return None, "Please select two songs."

    try:
        out_path = remix_two_core(song_a, song_b, g_vocals, g_drums, g_bass, g_other)
    except Exception as e:
        return None, f"Error during remix: {e}"

    info = (
        f"Song A (vocals & bass): {song_a}\n"
        f"Song B (drums & other): {song_b}\n"
        f"Gains: vocals={g_vocals:.2f}, drums={g_drums:.2f}, "
        f"bass={g_bass:.2f}, other={g_other:.2f}"
    )
    return out_path, info


def random_two_song_remix():
    # choose two distinct songs
    idx = np.random.choice(len(SONG_LIST), size=2, replace=False)
    song_a = SONG_LIST[idx[0]]
    song_b = SONG_LIST[idx[1]]

    g_vocals = float(np.random.uniform(0.7, 1.6))
    g_drums = float(np.random.uniform(0.7, 1.6))
    g_bass = float(np.random.uniform(0.7, 1.6))
    g_other = float(np.random.uniform(0.7, 1.6))

    try:
        out_path = remix_two_core(song_a, song_b, g_vocals, g_drums, g_bass, g_other)
    except Exception as e:
        return (
            None,
            f"Error during random two-song remix: {e}",
            song_a,
            song_b,
            g_vocals,
            g_drums,
            g_bass,
            g_other,
        )

    info = (
        "[Random two-song remix]\n"
        f"Song A (vocals & bass): {song_a}\n"
        f"Song B (drums & other): {song_b}\n"
        f"Gains: vocals={g_vocals:.2f}, drums={g_drums:.2f}, "
        f"bass={g_bass:.2f}, other={g_other:.2f}"
    )
    return out_path, info, song_a, song_b, g_vocals, g_drums, g_bass, g_other


with gr.Blocks() as demo:
    gr.Markdown("## MUSDB18HQ Two-Song Remix App (no upload needed)")

    gr.Markdown(
        f"Root folder: `{ROOT_DIR}`  \n"
        f"Found **{len(SONG_LIST)}** songs with 4 stems."
    )

    with gr.Row():
        song_a_dropdown = gr.Dropdown(
            choices=SONG_LIST,
            value=SONG_LIST[0],
            label="Song A (vocals & bass)",
        )
        song_b_dropdown = gr.Dropdown(
            choices=SONG_LIST,
            value=SONG_LIST[1],
            label="Song B (drums & other)",
        )

    gr.Markdown("### Gains (0 = mute, 1 = original volume, >1 = louder)")

    with gr.Row():
        g_vocals = gr.Slider(0, 2, value=1.0, step=0.05, label="Vocals (from A)")
        g_drums = gr.Slider(0, 2, value=1.0, step=0.05, label="Drums (from B)")
    with gr.Row():
        g_bass = gr.Slider(0, 2, value=1.0, step=0.05, label="Bass (from A)")
        g_other = gr.Slider(0, 2, value=1.0, step=0.05, label="Other (from B)")

    with gr.Row():
        btn_manual = gr.Button("Remix selected two songs")
        btn_random = gr.Button("Random two-song mix")

    audio_out = gr.Audio(label="Remixed audio", interactive=False, type="filepath")
    info_box = gr.Textbox(label="Remix info", lines=4)

    btn_manual.click(
        manual_two_song_remix,
        inputs=[song_a_dropdown, song_b_dropdown, g_vocals, g_drums, g_bass, g_other],
        outputs=[audio_out, info_box],
    )

    btn_random.click(
        random_two_song_remix,
        inputs=[],
        outputs=[
            audio_out,
            info_box,
            song_a_dropdown,
            song_b_dropdown,
            g_vocals,
            g_drums,
            g_bass,
            g_other,
        ],
    )

demo.launch(debug=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e0c1b440a0a2a9e3a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
